Smart Pointers
---

<iframe src="https://giphy.com/embed/3o7TKFy1gaoy1JNqZG" width="480" height="330" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/hulu-snl-saturday-night-live-nbc-3o7TKFy1gaoy1JNqZG">via GIPHY</a></p>

A pointer is a general concept for a variable that contains an address in memory.  
This address refers to, or “points at,” some other data. The most common kind of  
pointer in Rust is a reference, which you learned about in Chapter 4. References  
are indicated by the & symbol and borrow the value they point to. They don’t  
have any special capabilities other than referring to data, and have no overhead.  

Smart pointers, on the other hand, are data structures that act like a pointer  
but also have additional metadata and capabilities. The concept of smart  
pointers isn’t unique to Rust: smart pointers originated in C++ and exist in  
other languages as well. Rust has a variety of smart pointers defined in the  
standard library that provide functionality beyond that provided by references.  
To explore the general concept, we’ll look at a couple of different examples of  
smart pointers, including a reference counting smart pointer type. This pointer  
enables you to allow data to have multiple owners by keeping track of the number  
of owners and, when no owners remain, cleaning up the data.  

#### -- Smart Pointers own Data, References only Borrow --

Rust, with its concept of ownership and borrowing, has an additional difference  
between references and smart pointers: while references only borrow data, in  
many cases, smart pointers own the data they point to.  

Though we didn’t call them as such at the time, we’ve already encountered a few  
smart pointers in this book, including `String` and `Vec<T>` in Chapter 8. Both  
these types count as smart pointers because they own some memory and allow you  
to manipulate it. They also have metadata and extra capabilities or guarantees.  
String, for example, stores its capacity as metadata and has the extra ability  
to ensure its data will always be valid UTF-8.  

#### -- Structs with Deref and Drop Traits --

Smart pointers are usually implemented using structs. Unlike an ordinary struct,  
smart pointers implement the `Deref` and `Drop` traits. 
- The `Deref` trait allows an instance of the smart pointer struct to behave  
like a reference so you can write your code to work with either references or  
smart pointers.  
- The `Drop` trait allows you to customize the code that’s run when an instance  
of the smart pointer goes out of scope.  

In this chapter, we’ll discuss both traits and demonstrate why they’re important  
to smart pointers.

In [3]:
struct ToyRobot{
    name: String
}

impl Drop for ToyRobot{
    fn drop(&mut self){
        println!("{} has been DROPPED", self.name);
    }
}

{   // start playing with ToyRobot
    let my_robot = ToyRobot{name: String::from("Bender")};
    println!("{} is alive!", my_robot.name);
}   // done playing with ToyRobot
    // <- At this point the ToyRobot is dropped
    // "Bender has been DROPPED" will get printed before the end of the block

Bender is alive!
Bender has been DROPPED


()

#### -- Scope of Smart Pointer Pattern Covered Here --

Given that the smart pointer pattern is a general design pattern used frequently  
in Rust, this chapter won’t cover every existing smart pointer. Many libraries  
have their own smart pointers, and you can even write your own. We’ll cover the  
most common smart pointers in the standard library:  

- **`Box<T>`** for allocating values on the heap  
- **`Rc<T>`**, a reference counting type that enables multiple ownership  
- **`Ref<T>`** and **`RefMut<T>`**, accessed through **`RefCell<T>`**, a type  
that enforces the borrowing rules at runtime instead of compile time  


In addition, we’ll cover : 
- the interior mutability pattern where an immutable type exposes an API for  
mutating an interior value. 

We’ll also discuss reference cycles: 
- how they can leak memory and how to prevent them.  

Let’s dive in!  

Using Box<T> to Point to Data on the Heap
---

<iframe src="https://giphy.com/embed/8RH4MrJPTnVRK" width="369" height="480" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/guy-foam-stacking-8RH4MrJPTnVRK">via GIPHY</a></p>

The most straightforward smart pointer is a box, whose type is written `Box<T>`.  
Boxes allow you to store data on the heap rather than the stack. What remains on  
the stack is the pointer to the heap data. Refer to Chapter 4 to review the  
difference between the stack and the heap.  

Boxes don’t have performance overhead, other than storing their data on the heap  
instead of on the stack. But they don’t have many extra capabilities either.  
You’ll use them most often in these situations: 

- When you have a `type whose size can’t be known at compile time` and  
you want to use a value of that type in a context that requires an exact size  
- When you have a large amount of data and you want to `transfer ownership` but  
`ensure the data won’t be copied` when you do so  
- When you want to own a value and you care only that it’s a type that  
implements a particular trait rather than being of a specific type **`????`**

We’ll demonstrate the first situation in the  
“Enabling Recursive Types with Boxes” section. In the second case, transferring  
ownership of a large amount of data can take a long time because the data is  
copied around on the stack. To improve performance in this situation, we can  
store the large amount of data on the heap in a box. Then, only the small amount  
of pointer data is copied around on the stack, while the data it references  
stays in one place on the heap. The third case is known as a trait object, and  
Chapter 17 devotes an entire section, “Using Trait Objects That Allow for Values  
of Different Types,” just to that topic. So what you learn here you’ll apply  
again in Chapter 17!

### Using a Box<T> to Store Data on the Heap

Before we discuss the heap storage use case for Box<T>, we’ll cover the syntax  
and how to interact with values stored within a Box<T>.

Listing 15-1 shows how to use a box to store an i32 value on the heap:  

> Filename: src/main.rs


In [7]:
// the closure is defined and pushed to the stack
{   // closure begin
    
    let b = Box::new(5);   // the value 5 is allocated on the heap
                           // `b `is a box pointer allocated on the stack 
                           // pointing to the heap location where value 5 is 
                           // stored (it's address)
    println!("b = {}", b);  // println! is accessing 'b' and automatccally 
                            // dereferencing it to access the value 5
                            // @audit : why didn't we need to dereference `b`?
                            // ANSWER : println! the macro automatically 
                            // dereferences the box pointer `b` to access the
                            // value 5 stored on the heap
    // - storing a single compile time known value on the heap is not very useful
    // - storing on the heap is more useful in later examples when we store much
    // LARGER and VARIABLE amounts of data on the heap

    let c = *b + 2; // Here we are manually dereferencing the box pointer `b`
                    // to access the value 5 stored on the heap
    println!("c = {}", c);

}   // closure ends
// the Box pointer `b` goes out of scope and is deallocated
// the heap memory address where the value 5 is stored is deallocated
// closure is popped off the stack

b = 5


c = 7


> Listing 15-1: Storing an i32 value on the heap using a box

#### -- Line by Line Stack and Heap op for Box Declaration --

We define the variable b to have the value of a Box that points to the value 5,  
which is allocated on the heap. This program will print b = 5; in this case, we  
can access the data in the box similar to how we would if this data were on the  
stack. Just like any owned value, when a box goes out of scope, as b does at the  
end of main, it will be deallocated. The deallocation happens both for the box  
(stored on the stack) and the data it points to (stored on the heap).  

Putting a single value on the heap isn’t very useful, so you won’t use boxes by  
themselves in this way very often. Having values like a single i32 on the stack,  
where they’re stored by default, is more appropriate in the majority of  
situations. Let’s look at a case where boxes allow us to define types that we  
wouldn’t be allowed to if we didn’t have boxes.

### Enabling Recursive Types with Boxes

A value of recursive type can have another value of the same type as part of  
itself. Recursive types pose an issue because at compile time Rust needs to know  
how much space a type takes up. However, the nesting of values of recursive  
types could theoretically continue infinitely, so Rust can’t know how much space  
the value needs. Because boxes have a known size, we can enable recursive types  
by inserting a box in the recursive type definition.  

As an example of a recursive type, let’s explore the cons list. This is a data  
type commonly found in functional programming languages. The cons list type  
we’ll define is straightforward except for the recursion; therefore, the  
concepts in the example we’ll work with will be useful any time you get into  
more complex situations involving recursive types.  

#### -- Explain Recursion with the Cons List --

A cons list is a data structure that comes from the Lisp programming language  
and its dialects and is made up of nested pairs, and is the Lisp version of a  
linked list. Its name comes from the cons function (“construct function”) in  
Lisp that constructs a new pair from its two arguments. By calling cons on a  
pair consisting of a value and another pair, we can construct cons lists made up  
of recursive pairs.  

#### -- Cons List Pseudo Code Example --

For example, here’s a pseudocode representation of a cons list containing the  
list 1, 2, 3 with each pair in parentheses:  

```rust
(1, (2, (3, Nil)))
```

Each item in a cons list contains two elements: the value of the current item  
and the next item. The last item in the list contains only a value called Nil  
without a next item. A cons list is produced by recursively calling the cons  
function. The canonical name to denote the base case of the recursion is Nil.  
Note that this is not the same as the “null” or “nil” concept in Chapter 6,  
which is an invalid or absent value.

> The cons list isn’t a commonly used data structure in Rust. Most of the time  
when you have a list of items in Rust, Vec<T> is a better choice to use. Other,  
more complex recursive data types are useful in various situations, but by  
starting with the cons list in this chapter, we can explore how boxes let us  
define a recursive data type without much distraction.  

#### -- Cons List Implemented as an Enum Definition --

Listing 15-2 contains an enum definition for a cons list. Note that this code  
won’t compile yet because the List type doesn’t have a known size, which we’ll  
demonstrate.

> Filename: src/main.rs

In [8]:
enum List { // Recursive type List has INFINITE size
    // Recursive without indirection
    Cons(i32, List),
    Nil,
}

Error: recursive type `List` has infinite size

#### -- Compile Error : Recursion Without Indirection -- 

> Listing 15-2: The first attempt at defining an enum to represent a cons list  
data structure of i32 values

```lua
Note: We’re implementing a cons list that holds only i32 values for the purposes  
of this example. We could have implemented it using generics, as we discussed in  
Chapter 10, to define a cons list type that could store values of any type.
```

Using the List type to store the list 1, 2, 3 would look like the code in  
Listing 15-3:

> Filename: src/main.rs

The first Cons value holds 1 and another List value. This List value is another  
Cons value that holds 2 and another List value. This List value is one more Cons  
value that holds 3 and a List value, which is finally Nil, the non-recursive  
variant that signals the end of the list.  

If we try to compile the code in Listing 15-3, we get the error shown in  
Listing 15-4:  

```sh
$ cargo run
   Compiling cons-list v0.1.0 (file:///projects/cons-list)
error[E0072]: recursive type `List` has infinite size
 --> src/main.rs:1:1
  |
1 | enum List {
  | ^^^^^^^^^
2 |     Cons(i32, List),
  |               ---- recursive without indirection
  |
help: insert some indirection (e.g., a `Box`, `Rc`, or `&`) to break the cycle
  |
2 |     Cons(i32, Box<List>),
  |               ++++    +

For more information about this error, try `rustc --explain E0072`.  
error: could not compile `cons-list` due to previous error  
```

> Listing 15-4: The error we get when attempting to define a recursive enum

In [7]:
// Derive the Debug trait for pretty printing the List enum using {:?} specifier
#[derive(Debug)]    // Rust will GENERATE the Debug Trait for enum List
enum List {         // Define an enum with 2 variants
                    // - Cons
                    // - Nil
    // Using Box<> here avoids the infinite recursion size issue because
    // Box<List> is a heap allocated value that allows us to store another List
    // - Instead of potentially expanding to infinite size of the Cons List,
    // We are recursively writing and retrieving from the defined address on the
    // heap as we process the List
    // @audit-ok : Time Recursion on Fixed Space -> One shot on Unbounded Space?
    // ANSWER : No, it's not a recursion on fixed space, it's just recursion on
    // the heap and the heap is LARGER than the stack so the recursion can go
    // DEEPER, but it's not actually INFINITE 
    // It's BOUNDED to the PHYSICAL memory of our compute SUBSTRATE
    Cons(i32, Box<List>), 
    // Nil variant represents an empty list
    // Essentially the EOF (end of file) equivalent for Cons List
    Nil,
}

impl Drop for List {
    fn drop(&mut self){
        match self{
            Cons (value, next) => println!("Dropping Cons with value {} -> next {:?}", value, next),
            Nil => println!("Dropping Nil"),
        }
        // println!("Dropping {:?}", self);
    }
}

    // push new frame to the stack
{   // begin new scope

    // `list` comes into scope, creating a VARIABLE Cons List
    // - With 3 cons elements : 1 -> 2 -> 3 -> Nil
    // - Each cons element is followed by another List wrapped in a Box<>
    // - The last element is Nil which is the EOL (end-of-list) for the List
    // - While creating this list, we allocate memory for each Box<List>
    let list = List::Cons(1, Box::new(List::Cons(2, Box::new(List::Cons(3, Box::new(List::Nil))))));
    println!("list = {:?}", list);
}   // end scope
    
// `list` goes out of scope
// - Rust calls the drop function for each Box<List> in the list
// - this deallocates the heap memory for each Box<List>
// - Finally the stack memory used by the `list` variable is deallocated
// pop frame off the stack

list = Cons(1, Cons(2, Cons(3, Nil)))
Dropping Cons with value 1 -> next Cons(2, Cons(3, Nil))
Dropping Cons with value 2 -> next Cons(3, Nil)
Dropping Cons with value 3 -> next Nil
Dropping Nil


#### -- Cons List Fixed : Indirect to `Box<List>` recursion --

The error shows this type “has infinite size.” The reason is that we’ve defined  
List with a variant that is recursive: it holds another value of itself directly.  
As a result, Rust can’t figure out how much space it needs to store a List value.  
Let’s break down why we get this error. First, we’ll look at how Rust decides  
how much space it needs to store a value of a non-recursive type.  

### Computing the Size of a Non-Recursive Type

Recall the Message enum we defined in Listing 6-2 when we discussed enum  
definitions in Chapter 6:  

```rust
enum Message {
    Quit,
    Move { x: i32, y: i32 },
    Write(String),
    ChangeColor(i32, i32, i32),
}
```

To determine how much `space to allocate for a Message value`, Rust goes  
through each of the variants to `see which variant needs the most space`

Rust sees that :  

- Message::Quit doesn’t need any space,  
- Message::Move needs enough space to store two i32 values,  
- and so forth.  

Because only one variant will be used, the most space a Message value will need  
is the space it would take to store the largest of its variants.


#### -- Illustrate Recursion (-)Indirection : Compile Time Error --

Contrast this with what happens when Rust tries to determine how much space a  
recursive type like the List enum in Listing 15-2 needs. The compiler starts by  
looking at the Cons variant, which holds a value of type i32 and a value of type  
List. Therefore, Cons needs an amount of space equal to the size of an i32 plus  
the size of a List. To figure out how much memory the List type needs, the  
compiler looks at the variants, starting with the Cons variant. The Cons variant  
holds a value of type i32 and a value of type List, and this process continues  
infinitely, as shown in Figure 15-1.  

<img src="https://doc.rust-lang.org/book/img/trpl15-01.svg" width=50% height=50%>

> Figure 15-1: An infinite List consisting of infinite Cons variants


### Using Box<T> to Get a Recursive Type with a Known Size

Because Rust can’t figure out how much space to allocate for recursively defined  
types, the compiler gives an error with this helpful suggestion:  

```sh
help: insert some indirection (e.g., a `Box`, `Rc`, or `&`) to make `List` representable
  |
2 |     Cons(i32, Box<List>),
  |               ++++    +
```

In this suggestion, “indirection” means that instead of **storing a value directly**,  
we should change the data structure to store the value indirectly by **storing a  
pointer to the value** instead.  

#### -- `Box<T>` the pointer is Fixed Size regardless of Data --

Because a Box<T> is a pointer, Rust always knows how much space a Box<T> needs:  

- a pointer’s size doesn’t change based on the amount of data it’s pointing to.  

This means we can put a Box<T> inside the Cons variant instead of another List  
value directly. The Box<T> will point to the next List value that will be on the  
heap rather than inside the Cons variant.  

Conceptually, we still have a list, created with lists holding other lists, but  
this implementation is now more like placing the items next to one another  
rather than inside one another.  

> This implementation solves calculating the size for the List enum :
```rust
enum List{
    Cons(i32, Box<List>),   // i32 == 4 bytes (32 bits)
                            // Box<List> == 4 bytes (32 bits)
                            // 8 bytes total (if system is 32 bits)
    Nil,                    // Doesn't need any space
}

```

We can change the definition of the List enum in Listing 15-2 and the usage of  
the List in Listing 15-3 to the code in Listing 15-5, which will compile:  

> Filename: src/main.rs

In [2]:
use crate::List::{Cons, Nil};

#[derive(Debug)]
enum List{
    Cons(i32, Box<List>),
    Nil,
}

impl Drop for List{
    fn drop(&mut self){
        println!("Dropping List: {:?}", self);
    }
}

{
    let list = Cons(1, Box::new(Cons(2, Box::new(Cons(3, Box::new(Nil))))));
    println!("Box<List> list = {:?}", list);
}

Box<List> list = Cons(1, Cons(2, Cons(3, Nil)))
Dropping List: Cons(1, Cons(2, Cons(3, Nil)))
Dropping List: Cons(2, Cons(3, Nil))
Dropping List: Cons(3, Nil)
Dropping List: Nil


()

***
###### -- [GPT Ask] --
***

> Dropping List: Cons(1, Cons(2, Cons(3, Nil)))  
Dropping List: Cons(2, Cons(3, Nil))  
Dropping List: Cons(3, Nil)  
Dropping List: Nil  

> Explain the drop ORDER ^

In the given code, the `List` enum is created using the `Cons` variant, which  
contains an `i32` value and a `Box<List>`. The `list` is built using nested  
`Cons` variants, each containing a boxed tail of the list:

```rust
let list = Cons(1, Box::new(Cons(2, Box::new(Cons(3, Box::new(Nil))))));
```

This `list` can be visualized as follows:

```rust
Cons(1) -> Cons(2) -> Cons(3) -> Nil
```

The `Drop` trait is implemented for the `List` enum. When the `list` goes out of  
scope at the end of the `main` function, the `drop` method is called for each  
element of the list, starting from the outermost `Cons` variant and working its  
way inwards.

The drop order can be understood by analyzing how the list is stored in memory  
and how the ownership of the boxed tails works:

1. When `list` goes out of scope, the `drop` method is called for the outermost  
`Cons` variant containing the value `1`. This prints the message "Dropping Cons  
with value 1". The ownership of the boxed tail (Cons(2, Cons(3, Nil))) is  
transferred to the `drop` method.

2. Now, the `drop` method is called for the next `Cons` variant containing the  
value `2`. This prints the message "Dropping Cons with value 2". The ownership  
of the boxed tail (Cons(3, Nil)) is transferred to the `drop` method.

3. The `drop` method is called for the next `Cons` variant containing the value  
`3`. This prints the message "Dropping Cons with value 3". The ownership of the  
boxed tail (Nil) is transferred to the `drop` method.

4. Finally, the `drop` method is called for the `Nil` variant. This prints the  
message "Dropping Nil".

The drop order follows the nested structure of the list, as each `Cons` element  
owns a boxed tail, and the drop method is called recursively for each owned  
tail. As a result, the drop order is:

```
Dropping List: Cons(1, Cons(2, Cons(3, Nil)))
Dropping List: Cons(2, Cons(3, Nil))
Dropping List: Cons(3, Nil)
Dropping List: Nil
```

> Listing 15-5: Definition of List that uses Box<T> in order to have a known size

#### -- Box breaks Recursive Chain => Indexed Chain --

The Cons variant needs the size of an i32 plus the space to store the box’s  
pointer data. The Nil variant stores no values, so it needs less space than the  
Cons variant. We now know that any List value will take up the size of an i32  
plus the size of a box’s pointer data. By using a box, we’ve broken the  
infinite, recursive chain, so the compiler can figure out the size it needs to  
store a List value. Figure 15-2 shows what the Cons variant looks like now.

<img src="https://doc.rust-lang.org/book/img/trpl15-02.svg" width=33%, height=33%>

> Figure 15-2: A List that is not infinitely sized because Cons holds a Box

#### -- Illustrate Recursion (+) Indirection : Compiles Ok --

Boxes provide only the indirection and heap allocation; they don’t have any  
other special capabilities, like those we’ll see with the other smart pointer  
types. They also don’t have the performance overhead that these special  
capabilities incur, so they can be useful in cases like the cons list where the  
indirection is the only feature we need. We’ll look at more use cases for boxes  
in Chapter 17, too.

The Box<T> type is a smart pointer because it implements the Deref trait,  
which allows Box<T> values to be treated like references. When a Box<T> value  
goes out of scope, the heap data that the box is pointing to is cleaned up as  
well because of the Drop trait implementation. These two traits will be even  
more important to the functionality provided by the other smart pointer types  
we’ll discuss in the rest of this chapter. Let’s explore these two traits in  
more detail.

Treating Smart Pointers Like Regular References with the `Deref` Trait
---

<iframe src="https://giphy.com/embed/8PDYiS12B6iYg" width="480" height="366" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/karate-concrete-chopping-8PDYiS12B6iYg">via GIPHY</a></p>

Implementing the `Deref` trait allows you to customize the behavior of the  
dereference operator `*` (not to be confused with the multiplication or glob  
operator). By implementing `Deref` in such a way that a smart pointer can be  
treated like a regular reference, you can write code that operates on references  
and use that code with smart pointers too.

Let’s first look at how the dereference operator works with regular references.  
Then we’ll try to define a custom type that behaves like Box<T>, and see why the  
dereference operator doesn’t work like a reference on our newly defined type.  
We’ll explore how implementing the `Deref` trait makes it possible for smart  
pointers to work in ways similar to references. Then we’ll look at Rust’s deref  
coercion feature and how it lets us work with either references or smart  
pointers.

> Note: there’s one big difference between the MyBox<T> type we’re about to  
build and the real Box<T>: our version will not store its data on the heap. We  
are focusing this example on Deref, so where the data is actually stored is less  
important than the pointer-like behavior.


### Following the Pointer to the Value

A regular `reference` is a type of pointer, and one way to think of a pointer is  
as an `arrow to a value stored somewhere` else. In Listing 15-6, we create a  
`reference` to an `i32` value and then use the `dereference` operator to  
`follow` the `reference` to the `value`:

> Filename: src/main.rs

In [8]:
{
    let x = 5;   // x is assigned 5
    let y = &x;  // y is assigned the address of x

    assert_eq!(5, x);  // 5 is the value at x
    assert_eq!(5, *y); // 5 is the value we get FOLLOWING where y is pointed to
    // assert_eq!(5, y);  // compilation error : 5 and y (a pointer) are different types
}

()

#### -- Line by Line Assert Pointer, Dereference and their Value --

Line by line explanation :

- The variable x holds an i32 `value` 5. 
- We set y equal to a `reference` to x. 

We can assert that : 
- x is equal to 5.  

However, if we want to make an assertion about the value in y :  
- we have to use `*y` to `follow the reference to the value` it’s pointing  
to (hence dereference) so the compiler can compare the actual value. 

Once we `dereference y`, we have access to the integer value y is pointing to : 
- we can `compare with 5`

If we tried to write assert_eq!(5, y); instead, we would get this compilation  
error:

```sh
$ cargo run
   Compiling deref-example v0.1.0 (file:///projects/deref-example)
error[E0277]: can't compare `{integer}` with `&{integer}`
 --> src/main.rs:6:5
  |
6 |     assert_eq!(5, y);
  |     ^^^^^^^^^^^^^^^^ no implementation for `{integer} == &{integer}`
  |
  = help: the trait `PartialEq<&{integer}>` is not implemented for `{integer}`
  = help: the following other types implement trait `PartialEq<Rhs>`:
            f32
            f64
            i128
            i16
            i32
            i64
            i8
            isize
          and 6 others
  = note: this error originates in the macro `assert_eq` (in Nightly builds,  
  run with -Z macro-backtrace for more info)

For more information about this error, try `rustc --explain E0277`.
error: could not compile `deref-example` due to previous error
```

> Comparing a number and a reference to a number isn’t allowed because they’re  
different types. We **must use the dereference operator to follow** the reference  
to the **value it’s pointing to**.

### Using Box<T> Like a Reference

We can rewrite the code in Listing 15-6 to use a Box<T> instead of a reference;  
the dereference operator used on the Box<T> in Listing 15-7 functions in the  
same way as the dereference operator used on the reference in Listing 15-6:

> Filename: src/main.rs

In [9]:
{
    let x = 5;
    let y = Box::new(x); // y is a Box containing the value of x

    assert_eq!(5, x);   // x still has it's original value
    assert_eq!(5, *y);  // *y dereferences the Box, following the pointer to 
                        // access the underlying value
                        // The deref operation goes through MULTIPLE levels:
                        // y -> *Box -> *x -> 5

    let z = Box::new(5);
    assert_eq!(5, *z);  // Since z contains a Box with the value 5 directly ...
                        // dereferencing takes less steps than y :
                        // z -> *Box -> 5
}

()

> Listing 15-7: Using the dereference operator on a Box<i32>

#### -- Difference between Box and Reference Example --

The main difference between Listing 15-7 and Listing 15-6 is that here we set y  
to be an instance of a Box<T> pointing to a copied value of x rather than a  
reference pointing to the value of x. In the last assertion, we can use the  
dereference operator to follow the pointer of the Box<T> in the same way that  
we did when y was a reference. Next, we’ll explore what is special about Box<T>  
that enables us to use the dereference operator by defining our own type.

***
###### -- [GPT Ask] --
***

> Can you explain the difference between a Box and a reference?

Certainly! Both `Box` and references (`&` and `&mut`) in Rust are used for  
working with data indirectly, but they serve different purposes and have  
different properties. Here's a brief explanation of the differences between a  
`Box` and a reference:

1. **Ownership and lifetime:** A `Box` is a smart pointer that owns the data it  
points to and is responsible for deallocating the memory when it goes out of  
scope. References, on the other hand, do not own the data they point to and do  
not deallocate memory. Instead, they "borrow" the data from the owner  
temporarily, and their lifetimes are bound to the scope of the owner.

2. **Heap allocation:** A `Box` always allocates its data on the heap, which is  
suitable for large or dynamically sized data structures. References do not  
allocate memory themselves; they point to data that may be stored on the stack  
or the heap, depending on the original owner.

3. **Mutability:** A `Box` allows mutable access to the data it owns if it is  
declared as mutable (e.g., `let mut x = Box::new(5)`). References can be either  
mutable (`&mut`) or immutable (`&`), but the mutable references follow Rust's  
borrowing rules, which prevent multiple mutable references or a mix of mutable  
and immutable references to the same data simultaneously. This ensures memory  
safety and prevents data races.

4. **Dereferencing:** Both `Box` and references can be dereferenced to access  
the underlying data. With a `Box`, you use the `*` operator to dereference  
(e.g., `*my_box`), and you do the same with references (e.g., `*my_ref`).

5. **Use cases:** A `Box` is often used for heap-allocated data, recursive data  
structures, or when transferring ownership of data between different parts of  
the code. References, on the other hand, are used for temporarily borrowing data  
without transferring ownership. This allows for more flexible and safe  
interactions with data, following Rust's borrowing rules.

In summary, a `Box` is a smart pointer that owns the data it points to and  
allocates it on the heap, while references are a way to temporarily borrow data  
without taking ownership. Both can be used to work with data indirectly, but  
their properties and use cases are different, depending on the needs of your  
program.

### Defining Our Own Smart Pointer

Let’s build a smart pointer similar to the Box<T> type provided by the standard  
library to experience how smart pointers behave differently from references by  
default. Then we’ll look at how to add the ability to use the dereference  
operator.

The Box<T> type is ultimately defined as a tuple struct with one element, so  
Listing 15-8 defines a MyBox<T> type in the same way. We’ll also define a new  
function to match the new function defined on Box<T>.

> Filename: src/main.rs

In [2]:
#[derive(Debug)]
struct MyBox<T>(T); // MyBox is a tuple struct with a single element ...

impl<T> MyBox<T>{
    fn new(x:T) -> MyBox<T>{
        MyBox(x)
    }
}

> Listing 15-8: Defining a MyBox<T> type

#### -- Minimum HomeBrew `MyBox<T>` Plain Struct Example --

We define a struct named MyBox and declare a generic parameter T, because we  
want our type to hold values of any type. The MyBox type is a tuple struct with  
one element of type T. The MyBox::new function takes one parameter of type T and  
returns a MyBox instance that holds the value passed in.  

Let’s try adding the main function in Listing 15-7 to Listing 15-8 and changing  
it to use the MyBox<T> type we’ve defined instead of Box<T>. The code in  
Listing 15-9 won’t compile because Rust doesn’t know how to dereference MyBox.  

> Filename: src/main.rs

In [3]:
{
    let x = 5;
    let y = MyBox::new(x);

    assert_eq!(5, x);
    assert_eq!(5, *y);  // error: type `MyBox<{integer}>` cannot be dereferenced
                        // To enable the * operator, we need to implement the Deref trait
}

Error: type `MyBox<{integer}>` cannot be dereferenced


> Listing 15-9: Attempting to use MyBox<T> in the same way we used references and Box<T>

Here’s the resulting compilation error:  

```sh
$ cargo run
   Compiling deref-example v0.1.0 (file:///projects/deref-example)
error[E0614]: type `MyBox<{integer}>` cannot be dereferenced
  --> src/main.rs:14:19
   |
14 |     assert_eq!(5, *y);
   |                   ^^

For more information about this error, try `rustc --explain E0614`.
error: could not compile `deref-example` due to previous error
```

#### -- Plain Struct doesn't Deref : Compilation Error --

Our MyBox<T> type can’t be dereferenced because we haven’t implemented that  
ability on our type. To enable dereferencing with the * operator, we implement  
the Deref trait.

### Treating a Type Like a Reference by Implementing the `Deref` Trait

As discussed in the “Implementing a Trait on a Type” section of Chapter 10, to  
implement a trait, we need to provide implementations for the trait’s required  
methods. The `Deref` trait, provided by the standard library, requires us to  
implement one method named deref that `borrows` self and `returns a reference` to  
the inner data. Listing 15-10 contains an implementation of Deref to add to the  
definition of MyBox:

Filename: src/main.rs

In [4]:
use std::ops::Deref;
use std::fmt::Debug; // Debug trait is required for println! macro

impl<T: Debug> Deref for MyBox<T>{ // Deref trait is implemented for MyBox<T>
    type Target = T;
    fn deref(&self) -> &Self::Target{
        // ... ah we constrained T to implement Debug trait to support this
        // println!() macro
        println!("Deref for MyBox<T> is called {:?}", self);
        &self.0 // @audit-ok : How come self is a tuple?
                // ANSWER : struct MyBox<T>(T); is a tuple struct with a single 
                // element ...
                // - self.0 is the first element of the tuple struct

    }
}

{ 
    let x = 5;
    let y = MyBox::new(x);

    assert_eq!(5, x);
    // deref for MyBox now works
    // @audit-ok - explain why does deref work for MyBox now
    // ANSWER : Because we implemented the Deref trait for MyBox
    assert_eq!(5, *y); 
}

// @audit-ok : why is there an error when I run this cell twice?
// ANSWER : We need to clear the previous implementation of Deref
// - REPL we need to reset manually -> Developer : Reload Window
// Running the cell twice in Jupyter produces an error due to CONFLICTING
// implementation of the Deref trait for MyBox<T>
// Jupyter stores the trait between cell execution so the previous Deref impl<>
// remains in sccope and conflicts with the new Deref impl<> 
// Fundamentally the impl<> must be unique

Deref for MyBox<T> is called MyBox(5)


> Listing 15-10: Implementing Deref on MyBox<T>

#### -- Implementing Deref on `MyBox<T>` --

- The type `Target = T;` syntax defines an associated type for the `Deref` trait  
to use. Associated types are a slightly different way of declaring a generic  
parameter, but you don’t need to worry about them for now; we’ll cover them in  
more detail in Chapter 19.  

- We fill in the body of the deref method with `&self.0` so deref returns a  
reference to the value we want to access with the * operator; recall from the  
“Using Tuple Structs without Named Fields to Create Different Types” section of  
Chapter 5 that `.0 accesses the first value in a tuple struct`. The main  
function in Listing 15-9 that calls * on the MyBox<T> value now compiles, and  
the assertions pass!

#### -- *(y.deref()) // * syntatic sugar => impl Deref --

Without the Deref trait, the compiler can only dereference & references. The  
deref method gives the compiler the ability to take a value of any type that  
implements Deref and call the deref method to get a & reference that it knows  
how to dereference.

When we entered *y in Listing 15-9, behind the scenes Rust actually ran this  
code:  

```rust
*(y.deref()) // syntatic sugar for *

// the plain dereference outside the parentheses in *(y.deref()) is still 
// necessary because if the deref method returned the value DIRECTLY the 
// - value would be moved out of self
// - we don't want OWNERSHIP of the inner value when DEREFERENCING in  
// most cases
// - *(deref()) pattern is a bias to Rust OWNERSHIP rules

```

Rust `substitutes the * operator with a call to the deref method`  
and then a plain dereference so we don’t have to think about whether or not we  
need to call the deref method. This Rust feature lets us write code that  
functions identically whether we have a regular reference or a type that  
implements Deref.

#### -- Ownership system requires indirect ref preserve self --

The reason the deref method returns a reference to a value, and that the plain  
dereference outside the parentheses in *(y.deref()) is still necessary, is to  
do with the ownership system. If the deref method returned the value directly  
instead of a reference to the value, the value would be moved out of self. We  
don’t want to take ownership of the inner value inside MyBox<T> in this case or  
in most cases where we use the dereference operator.  

> Note that the * operator is replaced with a call to the deref method and then  
a call to the * operator just once, each time we use a * in our code. Because  
the substitution of the * operator does not recurse infinitely, we end up with  
data of type i32, which matches the 5 in assert_eq! in Listing 15-9.  

### Implicit Deref Coercions with Functions and Methods

Deref coercion converts a reference to a type that implements the Deref trait  
into a reference to another type. 

For example, deref coercion can convert :

- `&String` to `&str` 
- because `String` implements the `Deref` trait such that it returns `&str` 

Deref coercion is a convenience Rust performs on arguments to functions and  
methods, and works only on types that implement the Deref trait. It happens  
automatically when we pass a reference to a particular type’s value as an  
argument to a function or method that doesn’t match the parameter type in the  
function or method definition. A sequence of calls to the deref method converts  
the type we provided into the type the parameter needs.

#### -- Deref coercion : &String => ownership &str => borrows --

Deref coercion was added to Rust so that programmers writing function and method  
calls don’t need to add as many explicit references and dereferences  
with & and *. The deref coercion feature also lets us write more code that can  
work for either references or smart pointers.

To see deref coercion in action, let’s use the MyBox<T> type we defined in  
Listing 15-8 as well as the implementation of Deref that we added in  
Listing 15-10. Listing 15-11 shows the definition of a function that has a  
string slice parameter:

> Filename: src/main.rs

In [5]:
fn hello(name: &str) {
    println!("Hello, {}!", name);
}

> Listing 15-11: A hello function that has the parameter name of type &str

#### -- hello(&str) Derefs to borrow string --

We can call the hello function with a string slice as an argument, such as  
hello("Rust"); for example. Deref coercion makes it possible to call hello with  
a reference to a value of type MyBox<String>, as shown in Listing 15-12:

> Filename: src/main.rs

In [7]:
{
    let m = MyBox::new(String::from("Rust"));
    hello(&m);  // @audit-ok : Line by line explanation of how MyBox is coerced to &str
                //
                // 1. &m is passed to the hello() function
                // 2. &m is a reference to MyBox<String>
                // 3. Since we implemented Deref for MyBox<T> where T is String
                // 4. Rust can turn &MyBox<String> into &String by calling deref()
                // 5. Rust can turn &String into &str by calling deref() again
                // 6. Rust calls hello() with &str which is what hello() expects
}

Deref for MyBox<T> is called MyBox("Rust")
Hello, Rust!


()

Listing 15-12: Calling hello with a reference to a MyBox<String> value, which  
works because of deref coercion

#### -- MyBox<String> deref coerced => &str : hello(&m) --

Here we’re calling the hello function with the argument &m, which is a reference  
to a MyBox<String> value. Because we implemented the Deref trait on MyBox<T> in  
Listing 15-10, Rust can turn &MyBox<String> into &String by calling deref. The  
standard library provides an implementation of Deref on String that returns a  
string slice, and this is in the API documentation for Deref. Rust calls deref  
again to turn the &String into &str, which matches the hello function’s definition.

If Rust didn’t implement deref coercion, we would have to write the code in  
Listing 15-13 instead of the code in Listing 15-12 to call hello with a value of  
type &MyBox<String>.

Filename: src/main.rs

In [42]:
{
    let m = MyBox::new(String::from("Rust"));
    // hello(&(m.deref()));
    hello(&(*m)[..]);
}

Deref for MyBox<T> is called MyBox("Rust")
Hello, Rust!


()

> Listing 15-13: The code we would have to write if Rust didn’t have deref  
coercion

#### -- (*m) then & and [..] take string slice // deref coercion --

The (*m) dereferences the MyBox<String> into a String. Then the & and [..] take  
a string slice of the String that is equal to the whole string to match the  
signature of hello. This code without deref coercions is harder to read, write,  
and understand with all of these symbols involved. Deref coercion allows Rust to  
handle these conversions for us automatically.

#### -- deref as many times needed to match parameter type at compile time --

When the Deref trait is defined for the types involved, Rust will analyze the  
types and use Deref::deref as many times as necessary to get a reference to  
match the parameter’s type. The number of times that Deref::deref needs to be  
inserted is resolved at compile time, so there is no runtime penalty for taking  
advantage of deref coercion!

### How Deref Coercion Interacts with Mutability

Similar to how you use the **`Deref`** trait to override **`the * operator`** on  
**`immutable references`**, you can use the **`DerefMut`** trait to  
**`override the * operator`** on **`mutable references`**.

Rust does deref coercion when it finds types and trait implementations in three  
cases:
- From &T to &U when T: Deref<Target=U>
- From &mut T to &mut U when T: DerefMut<Target=U>
- From &mut T to &U when T: Deref<Target=U>

The first two cases are the same as each other except that the second implements  
mutability. The first case states that if you have a &T, and T implements Deref  
to some type U, you can get a &U transparently. The second case states that the  
same deref coercion happens for mutable references.

#### -- Immutable Ref can NOT be COERCED to Mutable Ref

The third case is trickier: 

- Rust will also coerce a mutable reference to an immutable one. 
- But the reverse is not possible: immutable references will never coerce to  
mutable references. 

Because of the borrowing rules, if you have a mutable reference, that mutable  
reference must be the only reference to that data (otherwise, the program  
wouldn’t compile). 

Converting one mutable reference to one immutable reference will never break  
the borrowing rules. Converting an immutable reference to a mutable reference  
would require that the initial immutable reference is the only immutable  
reference to that data, but the borrowing rules don’t guarantee that. 

> Therefore, Rust can NOT make the assumption that converting an immutable  
reference to a mutable reference is possible.

Running Code on Cleanup with the `Drop` Trait
---

The second trait important to the smart pointer pattern is `Drop`, which lets you  
`customize what happens when a value is about to go out of scope`. You can provide  
an implementation for the `Drop` trait on any type, and that code can be used to  
`release resources` like `files` or `network connections`.

We’re introducing `Drop` in the context of smart pointers because the  
functionality of the `Drop` trait is almost always used when implementing a smart  
pointer. For example, when a `Box<T>` is dropped it will deallocate the space on  
the heap that the box points to.

#### -- Drop @ Scope End >>> Manually Tracked Drop --

<iframe src="https://giphy.com/embed/5Y4xPu6EjGuxFxVSC0" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mafs-married-at-first-sight-5Y4xPu6EjGuxFxVSC0">via GIPHY</a></p>

In some languages, for some types, the programmer must call code to free memory  
or resources every time they finish using an instance of those types. Examples  
include :  

- file handles,  
- sockets, or  
- locks.  

If they forget, the system might become overloaded and crash. In Rust, you can  
specify that a particular bit of code be run whenever a value goes out of scope,  
and the compiler will insert this code automatically. As a result, you don’t  
need to be careful about placing cleanup code everywhere in a program that an  
instance of a particular type is finished with—you still won’t leak resources!

#### -- Impl Drop trait => { // clean up code .. } --

You specify the code to run when a value goes out of scope by implementing the  
Drop trait. The `Drop` trait requires you to implement one method named drop that  
takes a mutable reference to self. To see when Rust calls drop, let’s implement  
drop with println! statements for now.

Listing 15-14 shows a `CustomSmartPointer `struct whose only custom functionality  
is that it will print `Dropping CustomSmartPointer!` when the instance goes out  
of scope, to show when Rust runs the drop function.

> Filename: src/main.rs

In [3]:
struct CustomSmartPointer{
    data: String,
}

impl Drop for CustomSmartPointer{
    fn drop(&mut self){
        println!("Dropping<Trait> CustomSmartPointer with data `{}`!", self.data);
    }
}
// @audit : Wait a minute... why am I able to rerun this impl cell without error?

// a real mature scenario lol
{
    let d = CustomSmartPointer{
        data: String::from("at the pool"),
    };
    let c = CustomSmartPointer{
        data: String::from("the kids off"),
    };
    // The order of droppage is the reverse of the order of creation
    println!("CustomSmartPointers created!");
}

CustomSmartPointers created!
Dropping<Trait> CustomSmartPointer with data `the kids off`!
Dropping<Trait> CustomSmartPointer with data `at the pool`!


()

Listing 15-14: A `CustomSmartPointer` struct that implements the `Drop` trait  
where we would put our cleanup code

#### -- Example cleanup code => { print @ drop } --

The `Drop` trait is included in the prelude, so we don’t need to bring it into  
scope. We implement the `Drop` trait on `CustomSmartPointer` and provide an  
implementation for the drop method that calls println!. The body of the drop  
function is where you would place any logic that you wanted to run when an  
instance of your type goes out of scope. We’re printing some text here to  
demonstrate visually when Rust will call drop.

#### -- Line by line tracking of scope and drop events --

In main, we create two instances of `CustomSmartPointer` and then print  
`CustomSmartPointers` created. At the end of main, our instances of  
`CustomSmartPointer` will go out of scope, and Rust will call the code we put  
in the drop method, printing our final message. Note that we didn’t need to  
call the drop method explicitly.

When we run this program, we’ll see the following output:

```sh
$ cargo run
   Compiling drop-example v0.1.0 (file:///projects/drop-example)
    Finished dev [unoptimized + debuginfo] target(s) in 0.60s
     Running `target/debug/drop-example`
CustomSmartPointers created!
Dropping<Trait> CustomSmartPointer with data `the kids off`!
Dropping<Trait> CustomSmartPointer with data `at the pool`!
```

Rust automatically called drop for us when our instances went out of scope,  
calling the code we specified.  

Variables are dropped in the reverse order of their creation, so :  

- `d` was dropped 
- before `c` 

> This example’s purpose is to give you a visual guide to how the drop method  
works; usually you would specify the cleanup code that your type needs to run  
rather than a print message.

### Dropping a Value Early with `std::mem::drop`

Unfortunately, it’s not straightforward to disable the automatic drop  
functionality. Disabling drop isn’t usually necessary : 

- The whole point of the **`Drop`** trait is that it’s taken care of automatically. 

Occasionally, however, you might want to clean up a value early. 

One example is when using smart pointers that manage locks: 

- You might want to force the drop method that releases the lock 
- So that other code in the same scope can acquire the lock. 

#### -- Disabling Automatic Drop is more complex --

Rust doesn’t let you call the Drop trait’s drop method manually; instead you  
have to call the **`std::mem::drop`** function provided by the standard  
library if you want to force a value to be dropped before the end of its scope.

If we try to call the **`Drop`** trait’s drop method manually by modifying the  
main function from Listing 15-14, as shown in Listing 15-15, we’ll get a  
compiler error:

> Filename: src/main.rs

In [21]:
{
    let c = CustomSmartPointer{
        data: String::from("Hot Potato"),
    };
    println!("CustomSmartPointer created.");
    c.drop(); // allowing this would cause a double free error
              // because when c goes out of scope, Rust will call drop() on it
              // ...again!
    println!("CustomSmartPointer dropped before the end of main.");
}

Error: explicit use of destructor method

> Listing 15-15: Attempting to call the drop method from the Drop trait manually  
to clean up early

When we try to compile this code, we’ll get this error:

```sh
$ cargo run
   Compiling drop-example v0.1.0 (file:///projects/drop-example)
error[E0040]: explicit use of destructor method
  --> src/main.rs:16:7
   |
16 |     c.drop();
   |     --^^^^--
   |     | |
   |     | explicit destructor calls not allowed
   |     help: consider using `drop` function: `drop(c)`

For more information about this error, try `rustc --explain E0040`.
error: could not compile `drop-example` due to previous error
```

#### -- Drop trait call => Destructor Compile Error --

This error message states that `we’re not allowed to explicitly call drop`.  
The error message uses the term destructor, which is the general programming  
term for a function that cleans up an instance. A destructor is analogous to a  
constructor, which creates an instance. The drop function in Rust is one  
particular destructor.

#### -- Double Free Error : if Drop trait are called manually --

Rust doesn’t let us call drop explicitly because Rust would still automatically  
call drop on the value at the end of main. This would cause a double free error  
because Rust would be trying to clean up the same value twice.

We can’t disable the automatic insertion of drop when a value goes out of scope,  
and we can’t call the drop method explicitly. So, if we need to force a value to  
be cleaned up early, we use the `std::mem::drop` function.

#### -- std::mem::drop is DIFFERENT from Drop trait --

The `std::mem::drop` function is `different from` the drop method in the  
`Drop` trait. We call it by passing as an argument the value we want to force  
drop. The function is in the prelude, so we can modify main in Listing 15-15 to  
call the drop function, as shown in Listing 15-16:

> Filename: src/main.rs

In [22]:
{
    let c = CustomSmartPointer{
        data: String::from("Hot Potato")
    };
    println!("CustomSmartPointer created.");
    drop(c); // using std::mem::drop
    println!("CustomSmartPointer dropped before the end of main.");
}

CustomSmartPointer created.
Dropping<Trait> CustomSmartPointer with data `Hot Potato`!
CustomSmartPointer dropped before the end of main.


()

> Listing 15-16: Calling std::mem::drop to explicitly drop a value before it  
goes out of scope

#### -- Line by line tracking of std::mem::drop example --

Running this code will print the following:

```sh
$ cargo run
   Compiling drop-example v0.1.0 (file:///projects/drop-example)
    Finished dev [unoptimized + debuginfo] target(s) in 0.73s
     Running `target/debug/drop-example`
CustomSmartPointer created.
Dropping<Trait> CustomSmartPointer with data `Hot Potato`!
CustomSmartPointer dropped before the end of main.
```

The text Dropping `CustomSmartPointer` with data `some data`! is printed  
between the `CustomSmartPointer` created. and `CustomSmartPointer` dropped  
before the end of main. text, showing that the `drop` method code is called to  
drop c at that point.

#### -- Drop trait : custom allocator built-in Ownership --

You can use code specified in a `Drop trait` implementation in many ways to  
make cleanup convenient and safe: for instance, you could use it to create your  
own memory allocator! With the `Drop trait` and Rust’s ownership system, you  
don’t have to remember to clean up because Rust does it automatically.

You also don’t have to worry about problems resulting from accidentally cleaning  
up values still in use: the ownership system that makes sure references are  
always valid also ensures that drop gets called only once when the value is no  
longer being used.

Now that we’ve examined `Box<T>` and some of the characteristics of smart  
pointers, let’s look at a few other smart pointers defined in the standard  
library.

`Rc<T>` The Reference Counted Smart Pointer
---

In the majority of cases, ownership is clear: you know exactly which variable  
owns a given value. However, there are cases when a single value might have  
multiple owners. For example, in graph data structures, multiple edges might  
point to the same node, and that node is conceptually owned by all of the edges  
that point to it. A node shouldn’t be cleaned up unless it doesn’t have any  
edges pointing to it and so has no owners.


#### -- When a VALUE needs MULTIPLE OWNERS --

You have to enable multiple ownership explicitly by using the Rust type Rc<T>,  
which is an abbreviation for reference counting. The Rc<T> type keeps track of  
the number of references to a value to determine whether or not the value is  
still in use. If there are zero references to a value, the value can be cleaned  
up without any references becoming invalid.

#### -- Rc<T> is like a SHARED TV in a FAMILY room --

<iframe src="https://giphy.com/embed/l2Je7CCBAFvpz5bfa" width="480" height="364" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/season-9-the-simpsons-9x20-l2Je7CCBAFvpz5bfa">via GIPHY</a></p>

Imagine Rc<T> as a TV in a family room. When one person enters to watch TV, they  
turn it on. Others can come into the room and watch the TV. When the last person  
leaves the room, they turn off the TV because it’s no longer being used. If  
someone turns off the TV while others are still watching it, there would be  
uproar from the remaining TV watchers!

#### -- Reference Count => unknown OWNER at compile time --

We use the Rc<T> type when we want to allocate some data on the heap for  
multiple parts of our program to read and we can’t determine at compile time  
which part will finish using the data last. If we knew which part would finish  
last, we could just make that part the data’s owner, and the normal ownership  
rules enforced at compile time would take effect.

> Note that `Rc<T> is only for` use in `single-threaded` scenarios. When we  
discuss concurrency in Chapter 16, we’ll cover how to do reference counting in  
multithreaded programs.

### Using Rc<T> to Share Data

Let’s return to our cons list example in Listing 15-5. Recall that we defined  
it using Box<T>. This time, we’ll create two lists that both share ownership of  
a third list. Conceptually, this looks similar to Figure 15-3:

<img src="https://doc.rust-lang.org/book/img/trpl15-03.svg">

> Figure 15-3: Two lists, b and c, sharing ownership of a third list, a

- We’ll create list a that contains `5` and then `10`. 
- Then we’ll make two more lists: 
    - b that starts with 3 and 
    - c that starts with 4. 
- Both b and c lists will then continue on to the first a list containing 5 and 10.  

In other words, both lists will share the first list containing 5 and 10.

Trying to implement this scenario using our definition of List with Box<T> won’t  
work, as shown in Listing 15-17:

> Filename: src/main.rs

In [3]:
enum List {
    Cons(i32, Box<List>),
    Nil,
}

// Import the variants of the List enum to avoid typing the FULL path every time
// - This is just a convenience to make the code more readable
// - Also we are OUTSIDE the enum scope, so we don't have LOCAL path access
use crate::List::{Cons, Nil};   // @audit-ok : why do we need to import this here?
                                // enum List def did NOT need it
                                // but the closure below DOES 
                                // ANSWER :
                                // Cons and Nil are variants of the List enum
                                // - their scope are automatically in the scope of
                                // the List enum
                                // @audit : Understand IMPLICIT scope of enum variants

{
    let a = Cons(5, Box::new(Cons(10, Box::new(Nil))));
    let _b = Cons(3, Box::new(a));  // 1st var to point to `a` - no error
    let _c = Cons(4, Box::new(a));  // errors when > 1 variable points to`a`
}
                                    // @audit : why???
                                    // ANSWER : 
                                    // - 1st `_b` assignment MOVES `a` into `_b`
                                    // - 2nd `_c` assignment references `a` var that 
                                    // has ALREADY been moved
                                    // - `a` can only have ONE owner at a time


Error: use of moved value: `a`


> Listing 15-17: Demonstrating we’re not allowed to have two lists using Box<T>  
that try to share ownership of a third list

When we compile this code, we get this error:

```sh
$ cargo run
   Compiling cons-list v0.1.0 (file:///projects/cons-list)
error[E0382]: use of moved value: `a`
  --> src/main.rs:11:30
   |
9  |     let a = Cons(5, Box::new(Cons(10, Box::new(Nil))));
   |         - move occurs because `a` has type `List`, which does not implement the `Copy` trait
10 |     let b = Cons(3, Box::new(a));
   |                              - value moved here
11 |     let c = Cons(4, Box::new(a));
   |                              ^ value used here after move

For more information about this error, try `rustc --explain E0382`.
error: could not compile `cons-list` due to previous error
```

#### -- Compile Error : Referencing data after MOVE  --

The Cons variants own the data they hold : 

- so when we create the `_b` list,  
- `a` is moved into `_b` and `_b` owns `a`. 

Then, when we try to use `a` again when creating `_c` : 
- we’re not allowed to because `a` has been moved

#### -- Changing Cons to hold References requires Lifetime --

We could change the definition of Cons to hold references instead, but then we  
would have to specify lifetime parameters. By specifying lifetime parameters, we  
would be specifying that every element in the list will live at least as long  
as the entire list.  

> This is the case for the elements and lists in Listing 15-17, but not in every  
scenario.

***
###### -- [GPT Ask] -- Holding Ref requires Lifetime
***

In Rust, ownership and borrowing are key concepts that help ensure memory  
safety. When dealing with data structures like linked lists, you might want to  
use references instead of owned values to save memory. However, using references  
introduces a new challenge: managing lifetimes.

Here's an explanation of the statement:

#### -- Changing Cons to hold References requires Lifetime --

Suppose you have a custom linked list called `Cons`. You want to change the  
definition of `Cons` to hold references instead of owned values.

```rust
enum List<'a, T> {
    Cons(&'a T, Box<List<'a, T>>),
    Nil,
}
```

In the above code snippet, we've changed `Cons` to hold a reference to `T`  
instead of an owned `T` value. Notice the use of the lifetime parameter `'a`.

By introducing the lifetime parameter `'a`, you are specifying that every  
element in the list (`&'a T`) will live at least as long as the entire list  
(`List<'a, T>`). This helps Rust's borrow checker to ensure that all references  
in the list are valid and won't lead to memory safety issues like  
"use after free."

In summary, changing `Cons` to hold references requires specifying lifetime  
parameters for the Rust borrow checker to ensure memory safety. These lifetime  
parameters ensure that every element in the list lives at least as long as the  
entire list.

#### -- Use RC :: Reference Count ~ ad hoc lifetime --

Instead, we’ll change our definition of `List` to use `Rc<T>` in place of  
`Box<T>`, as shown in Listing 15-18. Each `Cons` variant will now hold a `value`  
and an `Rc<T>` pointing to a `List`. 

- When we create `_b`, 
- instead of `taking ownership` of `a`, 
- we’ll `clone` the `Rc<List>` that `a` is holding, 
- thereby `increasing` the number of `references` from `one to two` and 
- letting `a` and `_b` share ownership of the data in that `Rc<List>`. 
- We’ll also `clone` `a` when creating `_c`, 
- `increasing` the number of `references` from `two to three`. 

Every time we call `Rc::clone`:  
- the `reference count` to the data within the `Rc<List>` will `increase`, and  
- the **`data won’t be cleaned up unless there are zero`** references to it.  

> Filename: src/main.rs

In [3]:
enum List {
    Cons(i32, Rc<List>),
    Nil,
}

// Rc<T> is not in the prelude, so we need to import it explicitly
// with these use statements
use crate::List::{Cons, Nil};
use std::rc::Rc;

{
    let a = Rc::new(Cons(5, Rc::new(Cons(10, Rc::new(Nil)))));
    // a.clone() also works ... @audit : tradeoff?
    // ANSWER :
    // - Rc::clone() is idiomatic Rust convention for advancing reference count
    // - Rc::clone() does NOT deep copy the data
    // - clone() can have an implementation that deep copies ... no guarantee
    // - so be default using Rc::clone() is GOOD FORM and allows us to have full
    // certainty that we are NOT DEEP COPYING, and narrows the surface area of
    // bugs and optimization efforts
    let _b = Cons(3, Rc::clone(&a));
    let _c = Cons(4, Rc::clone(&a));
}

()

Listing 15-18: A definition of List that uses Rc<T>  

We need to add a `use` statement to bring `Rc<T>` into scope because it’s not in  
the prelude. In main, we create the list holding 5 and 10 and store it in a new  
`Rc<List>` in `a`. Then when we create `_b` and `_c`, we call the `Rc::clone`  
function and `pass a reference` to the `Rc<List>` in a as an `argument`.

#### -- `Rc::clone(&a)` vs `a.clone()` tradeoffs --

We could have called `a.clone()` rather than `Rc::clone(&a)`, but :  

- Rust’s convention is to use `Rc::clone` in this case. 
- The implementation of `Rc::clone` doesn’t make a deep copy of all the data  
like most types’ implementations of clone do.  
- The call to `Rc::clone` only `increments the reference count`, which doesn’t  
take much time.  
- As opposed to Deep copies of data can take a lot of time.  

By using `Rc::clone` for reference counting, we can visually distinguish  
between:  

- the deep-copy kinds of clones and 
- the kinds of clones that increase the reference count.  

When looking for performance problems in the code, we only need to consider the  
deep-copy clones and can disregard calls to `Rc::clone`.

### Cloning an Rc<T> Increases the Reference Count

Let’s change our working example in Listing 15-18 so we can `see` the reference  
counts `changing` as we `create` and `drop` references to the `Rc<List>` in `_a`.

In Listing 15-19, we’ll change main so it has an inner scope around list `_c`;  
then we can see how the reference count changes when `_c` goes out of scope.

> Filename: src/main.rs

In [6]:
enum List {
    Cons(i32, Rc<List>),
    Nil,
}

use crate::List::{Cons, Nil};
use std::rc::Rc;

{
    let a = Rc::new(Cons(5, Rc::new(Cons(10, Rc::new(Nil)))));
    println!("count after creating a = {}", Rc::strong_count(&a));
    let _b = Cons(3, Rc::clone(&a));
    println!("count after creating b = {}", Rc::strong_count(&a));
    // @audit : weird that sometimes b prints before a ^ : why?
    // - the printed strong count values are CORRECT, but the order of printing
    // is RANDOM
    {
        let _c = Cons(4, Rc::clone(&a));
        println!("count after creating c = {} (inner scope)", Rc::strong_count(&a));
        // in this context a.clone() also works ...
        // - but Rc::clone() is idiomatic Rust convention for advancing reference 
        // count
        // - Rc::clone() does NOT deep copy the data
        let _z = Cons(3, a.clone());
        println!("count after creating z = {} (inner scope)", Rc::strong_count(&a));
    }
    println!("count after c goes out of scope = {} (return to main)", Rc::strong_count(&a));
}

count after creating a = 1
count after creating b = 2
count after creating c = 3 (inner scope)
count after creating z = 4 (inner scope)
count after c goes out of scope = 2 (return to main)


()

> Listing 15-19: Printing the reference count

#### -- `Rc::strong_count` to log reference changes --

At each point in the program where the reference count changes, we print the  
reference count, which we get by calling the `Rc::strong_count` function. This  
function is named `strong_count` rather than count because the `Rc<T>` type also  
has a weak_count; we’ll see what weak_count is used for in the  
“Preventing Reference Cycles: Turning an `Rc<T>` into a `Weak<T>`” section.  

This code prints the following:  

```sh
$ cargo run
   Compiling cons-list v0.1.0 (file:///projects/cons-list)
    Finished dev [unoptimized + debuginfo] target(s) in 0.45s
     Running `target/debug/cons-list`
count after creating a = 1
count after creating b = 2
count after creating c = 3
count after c goes out of scope = 2
```

We can see that the `Rc<List>` in a has :  
- an initial reference `count of 1`; 
- then each time we call `clone`, the `count goes up by 1`.  
- When `_c` goes `out of scope`, the count `goes down by 1`.  
- **`We don’t have to call a function to decrease the reference count`**  
like we have to call `Rc::clone` to increase the reference count: 
- the implementation of the `Drop` trait decreases the reference count  
automatically when an `Rc<T>` value goes out of scope.  

#### -- `Rc<>` : a value, multiple owner, count==lifetime -- 

What we can’t see in this example is that when `_b` and then `a` go out of scope  
at the end of main, the count is then 0, and the `Rc<List>` is cleaned up  
completely. Using `Rc<T>` allows a single value to have multiple owners, and  
the count ensures that the value remains valid as long as any of the owners  
still exist.

### -- `Rc<T>` works on IMMUTABLE REFERENCES ONLY --

Via immutable references, `Rc<T>` allows you to `share data` between multiple  
parts of your program for `reading only`. If `Rc<T>` allowed you to have  
multiple mutable references too, you might violate one of the borrowing rules  
discussed in Chapter 4: multiple mutable borrows to the same place can cause  
data races and inconsistencies. But being able to mutate data is very useful!  

In the next section, we’ll discuss the `interior mutability pattern` and the  
`RefCell<T>` type that you can use in conjunction with an `Rc<T>` to work with  
this immutability restriction.

`RefCell<T>` and the Interior Mutability Pattern
---

Interior mutability is a design pattern in Rust that allows you to mutate data  
even when there are immutable references to that data; normally, this action is  
disallowed by the borrowing rules. To mutate data, the pattern uses unsafe code  
inside a data structure to bend Rust’s usual rules that govern mutation and  
borrowing. Unsafe code indicates to the compiler that we’re checking the rules  
manually instead of relying on the compiler to check them for us; we will  
discuss unsafe code more in Chapter 19.

We can use types that use the interior mutability pattern only when we can :  

- ensure that the borrowing rules will be followed at runtime, 
- even though the compiler can’t guarantee that.  

The unsafe code involved is then wrapped in a safe API, and the outer type is  
still immutable.

Let’s explore this concept by looking at the `RefCell<T>` type that follows the  
interior mutability pattern.

### Enforcing Borrowing Rules at Runtime with `RefCell<T>`

Unlike `Rc<T>`, the `RefCell<T>` type represents single ownership over the data  
it holds. So, what makes `RefCell<T>` different from a type like `Box<T>`?  
Recall the borrowing rules you learned in Chapter 4:

- At any given time, you can have either (but not both) one mutable reference  
or any number of immutable references.
- References must always be valid.

With references and `Box<T>`, the borrowing rules’ invariants are enforced at  
compile time. With `RefCell<T>`, these invariants are enforced at runtime. With  
references, if you break these rules, you’ll get a compiler error.  
With `RefCell<T>`, if you break these rules, your program will panic and exit.

#### -- Borrowing Rule Checks at Compile vs RunTime --

The advantages of checking the borrowing rules at compile time are that errors  
will be caught sooner in the development process, and there is no impact on  
runtime performance because all the analysis is completed beforehand. For those  
reasons, checking the borrowing rules at compile time is the best choice in the  
majority of cases, which is why this is Rust’s default.

<iframe src="https://giphy.com/embed/f0BaErqmljUd2" width="480" height="420" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/pixel-glitch-f0BaErqmljUd2">via GIPHY</a></p>

#### -- runtime scenarios disallowed by compile check? --
The advantage of checking the borrowing rules at runtime instead is that certain  
memory-safe scenarios are then allowed, where they would’ve been disallowed by  
the compile-time checks. Static analysis, like the Rust compiler, is inherently  
conservative. Some properties of code are impossible to detect by analyzing the  
code: 

> the most famous example is the Halting Problem, which is beyond the scope of  
this book but is an interesting topic to research.

***
###### GPT ask : The Halting Problem
***

<iframe src="https://giphy.com/embed/3ohjUXdBJ1Yekkx1fy" width="480" height="360" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/heyarnold-3ohjUXdBJ1Yekkx1fy">via GIPHY</a></p>

> Explain the Halting Problem as if I am 5

Sure, here is the Halting Problem explained like you're 5:

Imagine you have a magical computer program that can run any other computer  
program. Someone gives you a new program and asks "Will this program ever stop  
running, or will it go on forever?"

You think, I have this magical computer, I'll just run the program and see if it  
stops!

So you run the program... and it runs... and runs... and you wait... but how  
will you know if it will EVER stop? You can't wait forever!

Even with your magical computer, there's no way to know for sure if that program  
will eventually stop or will run forever.

That's the halting problem - we can't write a single program that can determine  
if any other program will eventually stop running or continue endlessly.  
Computers have limitations on what they can figure out about other programs.

> Some programs halt, some loop endlessly, and in general there's no way to know  
the difference, no matter how clever our programming is.

That's the essence of the halting problem for a 5 year old! Let me know if you  
have any other questions!

#### -- Borrow guarantee : biased to REJECT v ACCEPT -- 

Because some analysis is impossible, if the Rust compiler can’t be sure the code  
complies with the ownership rules, it might reject a correct program; in this  
way, it’s conservative. If Rust accepted an incorrect program, users wouldn’t be  
able to trust in the guarantees Rust makes. However, if Rust rejects a correct  
program, the programmer will be inconvenienced, but nothing catastrophic can  
occur. The `RefCell<T>` type is useful when you’re sure your code follows the  
borrowing rules but the compiler is unable to understand and guarantee that.

Similar to `Rc<T>`, `RefCell<T>` is only for use in single-threaded scenarios  
and will give you a compile-time error if you try using it in a multithreaded  
context. We’ll talk about how to get the functionality of `RefCell<T>` in a  
multithreaded program in Chapter 16.

#### -- Table of `Box<T>`, `Rc<T>` and `RefCell<T>` features --

Here is a recap of the reasons to choose `Box<T>`, `Rc<T>`, or `RefCell<T>`:

- `Rc<T>` enables multiple owners of the same data; `Box<T>` and `RefCell<T>`  
have single owners.
- `Box<T>` allows `immutable or mutable` borrows checked at compile time;  
`Rc<T>`allows `only immutable` borrows checked at compile time; `RefCell<T>`  
allows immutable or mutable borrows `checked at runtime`.
- Because `RefCell<T>` allows mutable borrows `checked at runtime`, you can  
`mutate the value inside` the `RefCell<T>` even when the `RefCell<T>` is  
**`immutable`**.

Mutating the value inside an immutable value is the interior mutability pattern.  
Let’s look at a situation in which interior mutability is useful and examine how  
it’s possible.

### Interior Mutability : A Mutable Borrow to an Immutable Value

A consequence of the borrowing rules is that when you have an immutable value,  
you can’t borrow it mutably. For example, this code won’t compile:

In [7]:
{
    let x = 5;
    let _y = &mut x;
}

Error: cannot borrow `x` as mutable, as it is not declared as mutable

If you tried to compile this code, you’d get the following error:

```sh
$ cargo run
   Compiling borrowing v0.1.0 (file:///projects/borrowing)
error[E0596]: cannot borrow `x` as mutable, as it is not declared as mutable
 --> src/main.rs:3:13
  |
2 |     let x = 5;
  |         - help: consider changing this to be mutable: `mut x`
3 |     let y = &mut x;
  |             ^^^^^^ cannot borrow as mutable

For more information about this error, try `rustc --explain E0596`.
error: could not compile `borrowing` due to previous error
```

However, there are situations in which it would be useful for a value : 

- to mutate itself in its methods  
- but appear immutable to other code. 

#### -- Borrowing rules runtime check : **`HARD panic!`** --

Code outside the value’s methods would not be able to mutate the value. Using  
`RefCell<T>` is one way to get the ability to have interior mutability, but  
`RefCell<T>` doesn’t get around the borrowing rules completely:  

- the borrow checker in the compiler allows this interior mutability, and  
- the borrowing rules are checked at runtime instead.  

As a consequence, if you violate the rules, you’ll get a **`HARD panic!`**  
instead of a **`SOFT compiler error`**.

<iframe src="https://giphy.com/embed/Wr4shv0TxEVzO" width="480" height="353" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/Wr4shv0TxEVzO">via GIPHY</a></p>

Let’s work through a practical example where we can use `RefCell<T>` to mutate  
an immutable value and see why that is useful.

### A Use Case for Interior Mutability : Mock Objects

<iframe src="https://giphy.com/embed/3oKIPvFvTDFdtBzKc8" width="480" height="237" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/teamcoco-conan-obrien-quick-change-3oKIPvFvTDFdtBzKc8">via GIPHY</a></p>

Sometimes during testing a programmer will use a type in place of another type,  
in order to observe particular behavior and assert it’s implemented correctly.  
This placeholder type is called a test double. Think of it in the sense of a  
“stunt double” in filmmaking, where a person steps in and substitutes for an  
actor to do a particular tricky scene. Test doubles stand in for other types  
when we’re running tests. Mock objects are specific types of test doubles that  
record what happens during a test so you can assert that the correct actions  
took place.

Rust doesn’t have objects in the same sense as other languages have objects,  
and Rust doesn’t have mock object functionality built into the standard library  
as some other languages do. However, you can definitely `create a struct`  
that will `serve` the same purposes as a `mock object`.

Here’s the scenario we’ll test: we’ll create a library that tracks a value  
against a maximum value and sends messages based on how close to the maximum  
value the current value is. This library could be used to keep track of a user’s  
quota for the number of API calls they’re allowed to make, for example.

#### -- Mock Object Example : Message quota cap lib  --

Our library will only provide the functionality of tracking how close to the  
maximum a value is and what the messages should be at what times. Applications  
that use our library will be expected to provide the mechanism for sending the  
messages:  

- the application could put a message in the application,  
- send an email,  
- send a text message,  
- or something else.  

The library doesn’t need to know that detail. All it needs is something that  
`implements` a trait we’ll provide called `Messenger`.  

Listing 15-20 shows the library code:

> Filename: src/lib.rs

In [8]:
/// stub out this trait
/// This is the interface that :
/// - LimitTracker will use to call send()
/// - And also supports a mock implementation for testing
pub trait Messenger{
    fn send(&self, msg: &str);
}

/// stub out this struct
pub struct LimitTracker<'a, T:Messenger>{
    messenger: &'a T,
    value: usize,
    max: usize,
}

/// stub out this impl
impl<'a, T> LimitTracker<'a, T> 
where
    T:Messenger,
{
    /// create
    pub fn new(messenger:&'a T, max: usize) -> LimitTracker<'a, T>{
        LimitTracker{
            messenger,
            value: 0,
            max,
        }
    }
    /// update
    pub fn set_value(&mut self, value:usize){
        self.value = value;

        let percentage_of_max = self.value as f64 / self.max as f64;

        // match vs if else :
        // - more idiomatic in Rust to use match
        // - slightly less performant than if else because of pattern matching
        // - but in this case, that perf difference is negligible
        match () {
            _ if percentage_of_max >= 1.0 => {
                self.messenger.send("Error: you are over your quota!");
            }
            _ if percentage_of_max >= 0.9 => {
               self.messenger.send("Urgent warning: you've used up over 90% of your quota!");
            }
            _ if percentage_of_max >= 0.75 => {
                self.messenger.send("Warning: you've used up over 75% of your quota!");
            }
            _ => (),
        }
    }
}

> Listing 15-20: A library to keep track of how close a value is to a maximum  
value and warn when the value is at certain levels

One important part of this code is that the `Messenger` trait has one method  
called `send` that takes :  
- an immutable reference to self and 
- the text of the message.  

This trait is the interface our mock object needs to implement so that the mock  
can be used in the same way a real object is.  

#### -- `set_value` : how to assert on side effects no return ? --

The other important part is that we want to test the behavior of the `set_value`  
method on the `LimitTracker`. We can change what we pass in for the value  
parameter, but `set_value` doesn’t return anything for us to make assertions on.  
We want to be able to say that if we create a `LimitTracker` with something that  
implements the `Messenger` trait and a particular value for max, when we pass  
different numbers for value, the messenger is told to send the appropriate  
messages.

We need a mock object that, instead of sending an email or text message when we  
call `send`, will only keep track of the messages it’s told to send. We can : 

- create a new `instance` of the `mock object`, 
- create a `LimitTracker` that uses the `mock object`,  
- call the `set_value` method on `LimitTracker`,  
- and then `check` that the mock object has the `messages we expect`.  

#### -- Compiler Error: borrow obj mocked as mutable FAILS --

Listing 15-21 shows an attempt to implement a mock object to do just that, but  
the borrow checker won’t allow it:

> Filename: src/lib.rs

In [9]:
use std::cell::RefCell; // needed to support running test in Juptyer cell

//#[cfg(test)] // commenting this out to support running test in Juptyer cell
mod tests{
    use super::*; // @note : Ah test IS a great case for accessing parent scope

    // define a MockMessenger struct
    struct MockMessenger {
        // With a sent_messages field to capture send() calls
        // sent_messages: Vec<String>,
        sent_messages: RefCell<Vec<String>>, // RefCell is needed to support running test in Juptyer cell
    }

    impl MockMessenger {
        // new() as a convenient associated function to create a new MockMessenger
        fn new() -> MockMessenger {
            MockMessenger {
                // sent_messages: vec![],
                sent_messages: RefCell::new(vec![]), // RefCell is needed to support running test in Juptyer cell
            }
        }
    }

    // LimitTracker requires a Messenger trait, implement that for MockMessenger
    // so we can pass our MockMessenger into LimitTracker
    impl Messenger for MockMessenger {
        fn send(&self, message: &str) {
            // MockMessenger captures each message passed in send() 
            // Compile Error Here because
            // - &self is immutable
            // - but we are trying to mutate self.sent_messages
            // - we CAN NOT take the compiler suggestion to to use &mut self
            // because that would change the signature of MockMessenger.send()
            // to no longer match the signature of Messenger.send()
            // @note ... this is WHY we need INTERIOR MUTABILITY
            // self.sent_messages.push(String::from(message));
            self.sent_messages.borrow().push(String::from(message)); // borrow_mut is needed to support running test in Juptyer cell
        }
    }

    /*
    // commenting this out to support running test in Juptyer cell
    #[test]
    fn it_sends_an_over_75_percent_warning_message() {
    */
    /// test that LimitTracker sends a warning message at 75% of max
    pub fn it_sends_an_over_75_percent_warning_message() {
        // Create new MockMessenger with empty sent_messages vec[]
        let mock_messenger = MockMessenger::new();
        // Create new LimitTracker with max of 100 and pass our MockMessenger
        // object
        let mut limit_tracker = LimitTracker::new(&mock_messenger, 100);

        // set LimitTracker to a value greater than 75% of max : 80 > 75
        limit_tracker.set_value(80);

        // assert that at least one message was pushed to sent_messages vec[]
        // assert_eq!(mock_messenger.sent_messages.len(), 1);
        assert_eq!(mock_messenger.sent_messages.borrow().len(), 1); // borrow is needed to support running test in Juptyer cell
    }
}

Error: cannot borrow data in dereference of `Ref<'_, Vec<String>>` as mutable

In [10]:
// Run the test manually
tests::it_sends_an_over_75_percent_warning_message()

Error: failed to resolve: use of undeclared crate or module `tests`

> Listing 15-21: An attempt to implement a MockMessenger that isn’t allowed by  
the borrow checker

#### -- Review Mock Object test step by step --

This test code defines a `MockMessenger` struct that has a `sent_messages` field  
with a `Vec of String` values to keep track of the messages it’s told to send.  
We also define an `associated function new` to make it convenient to create new  
`MockMessenger` values that start with an empty list of messages. 

We then :  
- implement the `Messenger trait` for `MockMessenger` so we can give a  
`MockMessenger` to a `LimitTracker`. 
- In the definition of the `send` method, we take the message passed in as a  
parameter and store it in the `MockMessenger` list of `sent_messages`.

In the test, we’re testing what happens when the `LimitTracker` is told to set  
value to something that is more than 75 percent of the max value. 
- First, we create a new `MockMessenger`, which will start with an empty list of  
messages.  
- Then we create a new `LimitTracker` and give it a reference to the new  
`MockMessenger` and a max value of 100. 
- We call the `set_value` method on the `LimitTracker` with a value of `80`,  
which is more than 75 percent of 100. 
- Then we `assert` that the list of messages that the `MockMessenger` is keeping  
track of should now have `one message` in it.

However, there’s one problem with this test, as shown here:

```sh
$ cargo test
   Compiling limit-tracker v0.1.0 (file:///projects/limit-tracker)
error[E0596]: cannot borrow `self.sent_messages` as mutable, as it is behind a `&` reference
  --> src/lib.rs:58:13
   |
2  |     fn send(&self, msg: &str);
   |             ----- help: consider changing that to be a mutable reference: `&mut self`
...
58 |             self.sent_messages.push(String::from(message));
   |             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ `self` is a `&` reference, so the data it refers to cannot be borrowed as mutable

For more information about this error, try `rustc --explain E0596`.
error: could not compile `limit-tracker` due to previous error
warning: build failed, waiting for other jobs to finish...
```

- We can’t modify the `MockMessenger` to keep track of the messages, because the  
`send` method takes an `immutable reference to self`. 
- We also can’t take the suggestion from the error text to use `&mut self`  
instead, because then the signature of `send` wouldn’t match the signature in  
the `Messenger` trait definition 

(feel free to try and see what error message you get).

#### -- Here's where Interior Mutability comes in --

This is a situation in which interior mutability can help! We’ll store  
the `sent_messages` within a `RefCell<T>,` and then the `send` method will be  
able to modify sent_messages to store the messages we’ve seen.  
Listing 15-22 shows what that looks like:

> Filename: src/lib.rs

In [14]:
//#[cfg(test)] // commented out to manually run test in Jupyter cell
mod tests {
    use super::*;
    use std::cell::RefCell;

    struct MockMessenger {
        sent_messages: RefCell<Vec<String>>,
    }

    impl MockMessenger {
        fn new() -> MockMessenger {
            MockMessenger {
                sent_messages: RefCell::new(vec![]),
            }
        }
    }

    impl Messenger for MockMessenger {
        fn send(&self, message: &str){
            self.sent_messages.borrow_mut().push(String::from(message));
        }
    }

    //#[test] // commented out to manually run test in Jupyter cell
    //fn it_sends_an_over_75_percent_warning_message() {
    // set fn to public so we can manually run test in Jupyter cell
    pub fn it_sends_an_over_75_percent_warning_message() {
        let mock_messenger = MockMessenger::new();
        let mut limit_tracker = LimitTracker::new(&mock_messenger, 100);

        limit_tracker.set_value(80);

        assert_eq!(mock_messenger.sent_messages.borrow().len(), 1);
    }
}

In [12]:
// Run the test manually
tests::it_sends_an_over_75_percent_warning_message() // @audit : We don't we need a ; here?

> Listing 15-22: Using RefCell<T> to mutate an inner value while the outer value  
is considered immutable

The sent_messages field is now of type :  
- `RefCell<Vec<String>>` instead of `Vec<String>`.  
In the new function, we create a new `RefCell<Vec<String>>` instance around the  
empty vector.

For the implementation of the send method, the first parameter is still an  
immutable borrow of self, which matches the trait definition. We call  
`borrow_mut` on the RefCell<Vec<String>> in self.sent_messages to get a  
mutable reference to the value inside the RefCell<Vec<String>>, which is the  
vector. Then we can call push on the mutable reference to the vector to keep  
track of the messages sent during the test.

The last change we have to make is in the assertion: to see how many items  
are in the inner vector, we call borrow on the RefCell<Vec<String>> to get an  
immutable reference to the vector.

Now that you’ve seen how to use RefCell<T>, let’s dig into how it works!  

### Keeping Track of Borrows at Runtime with `RefCell<T>`

When creating immutable and mutable references, we use the & and &mut syntax,  
respectively. With RefCell<T>, we use the borrow and borrow_mut methods, which  
are part of the safe API that belongs to RefCell<T>. The borrow method returns  
the smart pointer type Ref<T>, and borrow_mut returns the smart pointer  
type RefMut<T>. Both types implement Deref, so we can treat them like regular  
references.

The RefCell<T> keeps track of how many Ref<T> and RefMut<T> smart pointers are  
currently active. Every time we call borrow, the RefCell<T> increases its count  
of how many immutable borrows are active. When a Ref<T> value goes out of scope,  
the count of immutable borrows goes down by one. Just like the compile-time  
borrowing rules, RefCell<T> lets us have many immutable borrows or one mutable  
borrow at any point in time.

If we try to violate these rules, rather than getting a compiler error as we  
would with references, the implementation of RefCell<T> will panic at runtime.  
Listing 15-23 shows a modification of the implementation of send in  
Listing 15-22. We’re deliberately trying to create two mutable borrows active  
for the same scope to illustrate that RefCell<T> prevents us from doing this at  
runtime.

> Filename: src/lib.rs

In [20]:
mod tests {
    use super::*;
    use std::cell::RefCell;

    struct MockMessenger {
        sent_messages: RefCell<Vec<String>>,
    }

    impl MockMessenger {
        fn new() -> MockMessenger {
            MockMessenger {
                sent_messages: RefCell::new(vec![]),
            }
        }
    }

    pub fn it_sends_an_over_75_percent_warning_message() {
        let mock_messenger = MockMessenger::new();
        let mut limit_tracker = LimitTracker::new(&mock_messenger, 100);

        limit_tracker.set_value(80);

        assert_eq!(mock_messenger.sent_messages.borrow().len(), 1);
    }

    impl Messenger for MockMessenger {
        fn send(&self, message: &str) {
            // @note : Will fail runtime borrow check because :
            // - we are borrowing 2 mutable references
            // borrow checker rule is that you can have either
            // - one mutable reference
            // - and/or any number of immutable references
            let mut one_borrow = self.sent_messages.borrow_mut();
            let mut two_borrow = self.sent_messages.borrow_mut();
    
            one_borrow.push(String::from(message));
            two_borrow.push(String::from(message));
        }
    }
}


In [21]:
// Run the test manually
tests::it_sends_an_over_75_percent_warning_message() 

thread '<unnamed>' panicked at 'already borrowed: BorrowMutError', src/lib.rs:88:53
stack backtrace:
   0: _rust_begin_unwind
   2: core::result::unwrap_failed
   3: <unknown>
   1: core::panicking::panic_fmt
   4: <unknown>
   5: <unknown>
   6: <unknown>
   7: evcxr::runtime::Runtime::run_loop
   8: evcxr::runtime::runtime_hook
   9: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


> Listing 15-23: Creating two mutable references in the same scope to see that  
RefCell<T> will panic

We create a variable one_borrow for the RefMut<T> smart pointer returned from  
borrow_mut. Then we create another mutable borrow in the same way in the  
variable two_borrow. This makes two mutable references in the same scope,  
which isn’t allowed. When we run the tests for our library, the code in  
Listing 15-23 will compile without any errors, but the test will fail:

```sh
$ cargo test
   Compiling limit-tracker v0.1.0 (file:///projects/limit-tracker)
    Finished test [unoptimized + debuginfo] target(s) in 0.91s
     Running unittests src/lib.rs (target/debug/deps/limit_tracker-e599811fa246dbde)

running 1 test
test tests::it_sends_an_over_75_percent_warning_message ... FAILED

failures:

---- tests::it_sends_an_over_75_percent_warning_message stdout ----
thread 'tests::it_sends_an_over_75_percent_warning_message' panicked at 'already borrowed: BorrowMutError', src/lib.rs:60:53
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


failures:
    tests::it_sends_an_over_75_percent_warning_message

test result: FAILED. 0 passed; 1 failed; 0 ignored; 0 measured; 0 filtered out; finished in 0.00s

error: test failed, to rerun pass `--lib`
```

Notice that the code panicked with the message already borrowed: BorrowMutError.  
This is how RefCell<T> handles violations of the borrowing rules at runtime.

Choosing to catch borrowing errors at runtime rather than compile time, as we’ve  
done here, means you’d potentially be finding mistakes in your code later in the  
development process: possibly not until your code was deployed to production.  
Also, your code would incur a small runtime performance penalty as a result of  
keeping track of the borrows at runtime rather than compile time. However,  
using RefCell<T> makes it possible to write a mock object that can modify itself  
to keep track of the messages it has seen while you’re using it in a context  
where only immutable values are allowed. You can use RefCell<T> despite its  
trade-offs to get more functionality than regular references provide.

### Having Multiple Owners of Mutable Data by Combining `Rc<T>` and `RefCell<T>`

A common way to use RefCell<T> is in combination with Rc<T>. Recall that  
Rc<T> lets you have multiple owners of some data, but it only gives immutable  
access to that data. If you have an Rc<T> that holds a RefCell<T>, you can  
get a value that can have multiple owners and that you can mutate!

For example, recall the cons list example in Listing 15-18 where we used  
Rc<T> to allow multiple lists to share ownership of another list. Because  
Rc<T> holds only immutable values, we can’t change any of the values in the  
list once we’ve created them. Let’s add in RefCell<T> to gain the ability to  
change the values in the lists. Listing 15-24 shows that by using a RefCell<T>  
in the Cons definition, we can modify the value stored in all the lists:

> Filename: src/main.rs

In [15]:
// Import these ... no implicit access in enum definition
use std::cell::RefCell;
use std::rc::Rc;

#[derive(Debug)]
enum List {
    Cons(Rc<RefCell<i32>>, Rc<List>),
    Nil,
}
    

{
    // enum implicit has access, so we didn't need to import at main scope
    // but since we will use it in this closure, manual import is needed
    use crate::List::{Cons, Nil};

    // Create an instance of Rc<RefCell<i32>> and store it in `value`
    let value = Rc::new(RefCell::new(5));

    // Create a List with a Cons variant that holds `value`
    // - we need to clone `value` so both `a` and `value` have ownership of the
    // 5, the value at the shared address
    // Rc::clone(&value) is so that we do NOT :
    // - move owner ship of `value` to `a`
    // - have `a` borrow `value`
    let a = Rc::new(Cons(Rc::clone(&value), Rc::new(Nil)));
    // Rc::clone(&a) for same reason as above
    let b = Cons(Rc::new(RefCell::new(3)), Rc::clone(&a));
    let c = Cons(Rc::new(RefCell::new(4)), Rc::clone(&a));

    // @audit : Explain borrow_mut in detail
    *value.borrow_mut() += 10;

    // print results
    println!("a after = {:?}", a);
    println!("b after = {:?}", b);
    println!("c after = {:?}", c);

}

a after = Cons(RefCell { value: 15 }, Nil)
b after = Cons(RefCell { value: 3 }, Cons(RefCell { value: 15 }, Nil))
c after = Cons(RefCell { value: 4 }, Cons(RefCell { value: 15 }, Nil))


()

> Listing 15-24: Using Rc<RefCell<i32>> to create a `List` that we can mutate

- We create a value that is an instance of `Rc<RefCell<i32>>` and store it in a  
variable named `value` so we can access it directly later.  
- Then we create a `List` in `a` with a `Cons` variant that holds `value`.  
- We need to clone `value` so both `a` and `value` have ownership of the  
`inner 5 value` rather than :  
    - transferring ownership from `value` to `a` or 
    - having `a` borrow from `value`.

> We wrap the list a in an `Rc<T>` so when we create lists `b` and `c`, they  
can both refer to `a`, which is what we did in Listing 15-18.

After we’ve created the lists in `a`, `b`, and `c`, we want to :  
- add `10` to the value in `value`.  
- We do this by calling `borrow_mut` on `value`, which uses the automatic  
dereferencing feature we discussed in Chapter 5  
(see the section “Where’s the -> Operator?”) to dereference the `Rc<T>` to the  
inner `RefCell<T>` value. 
- The `borrow_mut` method returns a `RefMut<T>` smart pointer, and we use the  
dereference operator on it and change the inner value.

When we print `a`, `b`, and `c`, we can see that they all have the modified  
value of `15` rather than `5`:

```sh
$ cargo run
   Compiling cons-list v0.1.0 (file:///projects/cons-list)
    Finished dev [unoptimized + debuginfo] target(s) in 0.63s
     Running `target/debug/cons-list`
a after = Cons(RefCell { value: 15 }, Nil)
b after = Cons(RefCell { value: 3 }, Cons(RefCell { value: 15 }, Nil))
c after = Cons(RefCell { value: 4 }, Cons(RefCell { value: 15 }, Nil))
```

This technique is pretty neat! By using `RefCell<T>`, we have :  
- an `outwardly immutable List` value.  
- But we can use the methods on `RefCell<T>` that provide access to its  
`interior mutability` so we can modify our data when we need to.  

The runtime checks of the borrowing rules protect us from data races, and it’s  
sometimes worth trading a bit of speed for this flexibility in our data  
structures. 

- Note that `RefCell<T>` does not work for multithreaded code!  
- `Mutex<T>` is the thread-safe version of `RefCell<T>` and we’ll discuss  
`Mutex<T>` in Chapter 16.

Reference Cycles Can Leak Memory
---

Rust’s memory safety guarantees make it difficult, but not impossible, to  
accidentally create memory that is never cleaned up (known as a memory leak).  
Preventing memory leaks entirely is not one of Rust’s guarantees, meaning  
memory leaks are memory safe in Rust. We can see that Rust allows memory leaks  
by using `Rc<T>` and `RefCell<T>`: it’s possible to create references where  
items refer to each other in a cycle. This creates memory leaks because the  
reference count of each item in the cycle will never reach 0, and the values  
will never be dropped.

### Creating a Reference Cycle

Let’s look at how a reference cycle might happen and how to prevent it,  
starting with the definition of the List enum and a tail method in  
Listing 15-25:

> Filename: src/main.rs

In [16]:
use std::cell::RefCell;
use std::rc::Rc;

#[derive(Debug)]
enum List {
    Cons(i32, RefCell<Rc<List>>),
    Nil,
}
    
use crate::List::{Cons, Nil};

impl List {
    fn tail(&self) -> Option<&RefCell<Rc<List>>> {
        match self {
            Cons(_, item) => Some(item),
            Nil => None,
        }
    }
}

> Listing 15-25: A cons list definition that holds a `RefCell<T>` so we can  
modify what a Cons variant is referring to

We’re using another variation of the List definition from Listing 15-5. The  
second element in the Cons variant is now `RefCell<Rc<List>>`, meaning that  
instead of having the ability to modify the `i32` value as we did in  
Listing 15-24, we want to modify the List value a Cons variant is pointing to.  
We’re also adding a tail method to make it convenient for us to access the  
second item if we have a Cons variant.

In Listing 15-26, we’re adding a main function that uses the definitions in  
Listing 15-25. This code creates a list in a and a list in b that points to  
the list in a. Then it modifies the list in a to point to b, creating a  
reference cycle. There are println! statements along the way to show what the  
reference counts are at various points in this process.

> Filename: src/main.rs

In [25]:
{
    let a = Rc::new(Cons(5, RefCell::new(Rc::new(Nil))));

    println!("a initial rc count = {}", Rc::strong_count(&a));  // 1
    println!("a next item = {:?}", a.tail()); // Some(RefCell { value: Nil })

    let b = Rc::new(Cons(10, RefCell::new(Rc::clone(&a))));

    println!("a rc count after b creation = {}", Rc::strong_count(&a)); // 2
    println!("b initial rc count = {}", Rc::strong_count(&b)); // 1
    println!("b next item = {:?}", b.tail());

    if let Some(link) = a.tail() {
        *link.borrow_mut() = Rc::clone(&b);
    }

    println!("b rc count after changing a = {}", Rc::strong_count(&b));
    println!("a rc count after changing a = {}", Rc::strong_count(&a));

    // Uncomment next line to see that we have a cycle;
    // it will overflow the stack
    println!("a next item = {:?}", a.tail());

}

a initial rc count = 1
a next item = Some(RefCell { value: Nil })
a rc count after b creation = 2
b initial rc count = 1
b next item = Some(RefCell { value: Cons(5, RefCell { value: Nil }) })
b rc count after changing a = 2
a rc count after changing a = 2
a next item = Some(RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { value: Cons(10, RefCell { value: Cons(5, RefCell { v

Error: Subprocess terminated with status: signal: 11 (SIGSEGV)